# Example use of rooted trees representations of product systems

### Introduction:
LCI databases built from unit process data can be very large. For example, ecoinvent v3.2 (cut-off) has 12,916 activities (unit processes), 116,248 technosphere flows (intermediary exchanges) and 343,020 biosphere exchanges (elementary flows).  
Consequently, the resulting product systems that are built for specific LCA that *use* the LCI databases are also significantly large.  
LCI databases built from unit process data also have cyclic dependency relationships, or *feedback loops*. For example, Electricity distribution is needed for aluminium production, and aluminium is needed for electricity distribution.  

There are cases where one could (1) want to reduce the size of one's product system, without losing any of the contributions to the total LCIA scores, or (2) have product systems where every instance of an activity is modeled as an independent unit process. Really, its true. This is why we did this.

Based on ideas presented in [Bourgault et al. 2012](http://link.springer.com/article/10.1007/s11367-012-0418-7), a "Tree" class was created that converts product systems modeled in the Brightway2 LCA framework into rooted tree representations of the same product systems.
  - The root is the unit process providing the functional unit.
  - All branches (i.e. internal nodes) are unit processes whose cradle-to-gate contributions are greater than a specified threshold (the "cutoff") for a given list of methods.
  - All other unit processes are considered leaves.
  - Leaves can be:
      * "entry points" to a disaggregated background LCI database
      * Datasets produced on the fly containing only LCIA scores
  - The resulting branch and leaf nodes are written to a new database.
  - The tree.root can then be used to create LCA objects as would normally be done in Brightway.

### Imports and set-up

In [76]:
# Import brightway2 LCA framework
from brightway2 import *

# Import the "Tree" class. Need to install as module or call from the right directory. 
from create_tree import Tree

# Import the augment_biosphere3 function, required to store leaves as LCIA results.
# See https://github.com/PascalLesage/my_bw_utils/blob/master/augment_biosphere3.py
# Need to install as module or call from the right directory. 
from my_bw_utils import augment_biosphere3 # Need to install as module or be in the right directory.

In [35]:
# Others
import time
import numpy as np

In [81]:
# Create a new project, set it up and import ecoinvent v3.2 cut-off:
projects.set_current("Tree examples")
bw2setup()
if 'ei32_CU_U' not in databases:
    ei = SingleOutputEcospold2Importer(r'C:\mypy\data\ecoinvent 3_2 CutOff\datasets', 'ei32_CU_U')
    ei.apply_strategies()
    ei.write_database()

Biosphere database already present!!! No setup is needed
Extracting XML data from 12916 datasets
Extracted 12916 datasets in 135.83 seconds
Applying strategy: normalize_units
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: nuncertainty
Applied 13 strategies in 3.82 seconds


Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:10:29


Title: Writing activities to SQLite3 database:
  Started: 09/28/2016 23:33:53
  Finished: 09/28/2016 23:44:23
  Total time elapsed: 00:10:29
  CPU %: 17.30
  Memory %: 12.29
Created database: ei32_CU_U


The storing of aggregated LCIA scores on leaves can only be done if the [biosphere database is first augmented](https://github.com/PascalLesage/my_bw_utils/blob/master/augment_biosphere3.py).

In [82]:
augment_biosphere3.addUnitImpactFlowsToBiosphereAndMethods()

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:06


Title: Writing activities to SQLite3 database:
  Started: 09/28/2016 23:46:15
  Finished: 09/28/2016 23:46:21
  Total time elapsed: 00:00:06
  CPU %: 22.70
  Memory %: 13.32


In [83]:
databases

Databases dictionary with 2 object(s):
	biosphere3
	ei32_CU_U

In [84]:
# Let's create a variable that references the ecoinvent database:
db = Database('ei32_CU_U')
print("There are {} activities in database {}".format(len(db), db.name))

There are 12916 activities in database ei32_CU_U


In [109]:
# Let's randomly select an activity from ecoinvent and an LCIA method:
random_activity = db.random()
random_method = methods.random()

## "Network LCA"
The typical Brightway2 usage would have you calculate LCA (LCI, LCIA, Monte-CArlo, what-have-you) using the full product system with its circular dependency relationships and all. This is normally quite fine, and would look something like this:

In [111]:
# Simple deterministic LCA
network_LCA = LCA({random_activity:1}, random_method)
network_LCA.lci()
network_LCA.lcia()
network_LCA.score

18.53145341756747

In [112]:
# Some statistics:)
print("The network product system has {} technosphere exchanges (i.e. elements in the A matrix)".format(len(network_LCA.tech_params)))
print("It also has has {} biosphere exchanges (i.e. non-null elements in the B matrix)".format(len(network_LCA.bio_params)))

The network product system has 116248 technosphere exchanges (i.e. elements in the A matrix)
It also has has 343020 biosphere exchanges (i.e. non-null elements in the B matrix)


## Trees with connections to the background
Let's create the tree with the cutoff pertaining to the random LCIA method + another random method:

In [113]:
list_of_LCIA_methods = [random_method, methods.random()]

In [114]:
# Intantiate the tree
tree_connected = Tree({random_activity:1})

# Traverse the tree
tree_connected.traverse(list_of_LCIA_methods,
                        "connected_tree",
                        cutoff=0.1,
                        store_leaves_as_scores=False)

# write the tree to the database "connected tree"
tree_connected.write_tree()

Writing activities to SQLite3 database:
0%       100%
[###########] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 09/28/2016 23:54:01
  Finished: 09/28/2016 23:54:02
  Total time elapsed: 00:00:01
  CPU %: 8.30
  Memory %: 18.04


We can now do the same operations are with the network LCA:

In [115]:
# Simple deterministic LCA
connected_tree_LCA = LCA({tree_connected.root:1}, random_method) #Explicitly calling the tree root
connected_tree_LCA.lci()
connected_tree_LCA.lcia()
connected_tree_LCA.score

18.53145363409269

In [116]:
# The result was also stored in the Tree object, along with the scores for all methods in the list passed to tree.traverse
tree_connected.scores[random_method]

18.531446901979397

In [117]:
# Some statistics:)
print("The network product system has {} technosphere exchanges (i.e. elements in the A matrix)".format(
        len(connected_tree_LCA.tech_params)))
print("It also has has {} biosphere exchanges (i.e. non-null elements in the B matrix)".format(
        len(connected_tree_LCA.bio_params)))

The network product system has 116405 technosphere exchanges (i.e. elements in the A matrix)
It also has has 343219 biosphere exchanges (i.e. non-null elements in the B matrix)


This is more than in the network model. This is because all branches and leaves are copies of datasets that also exist in the background:

In [118]:
print("There are {} more technosphere exchanges and {} more biosphere exchanges in this model than in the network equivalent".format(
        len(connected_tree_LCA.tech_params) - len(network_LCA.tech_params),
        len(connected_tree_LCA.bio_params) - len(network_LCA.bio_params)
      ))

There are 157 more technosphere exchanges and 199 more biosphere exchanges in this model than in the network equivalent


In [119]:
print("{} branch nodes and {} leaves were created".format(
        len(tree_connected.list_branches),
        len(tree_connected.list_leafs)
        ))

11 branch nodes and 136 leaves were created


The new database is dependent on the ecoinvent database:

In [124]:
Database('connected_tree').find_dependents()

['biosphere3', 'ei32_CU_U']

## Trees with leaves storing only cradle-to-gate impacts

In [120]:
# Intantiate the tree
tree_leaves_as_scores = Tree({random_activity:1})

# Traverse the tree
tree_leaves_as_scores.traverse(list_of_LCIA_methods,
                        "disconnected_tree",
                        cutoff=0.1,
                        store_leaves_as_scores=True)

# write the tree to the database "connected tree"
tree_leaves_as_scores.write_tree()

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 09/28/2016 23:54:54
  Finished: 09/28/2016 23:54:56
  Total time elapsed: 00:00:01
  CPU %: 28.20
  Memory %: 8.52


In [121]:
# Simple deterministic LCA
disconnected_tree_LCA = LCA({tree_leaves_as_scores.root:1}, random_method) #Explicitly calling the tree root
disconnected_tree_LCA.lci()
disconnected_tree_LCA.lcia()
disconnected_tree_LCA.score

18.531453580879443

In [125]:
# Some statistics:)
print("The network product system has {} technosphere exchanges (i.e. elements in the A matrix)".format(
        len(disconnected_tree_LCA.tech_params)))
print("It also has has {} biosphere exchanges (i.e. non-null elements in the B matrix)".format(
        len(disconnected_tree_LCA.bio_params)))

The network product system has 293 technosphere exchanges (i.e. elements in the A matrix)
It also has has 471 biosphere exchanges (i.e. non-null elements in the B matrix)


The number of flows is about three orders of magnitude smaller!

There are also no longer any links between the tree and the background database:

In [123]:
Database('disconnected_tree').find_dependents()

['biosphere3']